In [1]:
"""
Testing models and deciding on a final model
"""

# TODO: Test more models - RandomForestRegressor, XGBRegressor, CatBoostRegressor, LGBMRegressor
# TODO: Stacking of some kind (combine all - many examples in github).

''

In [1]:
import pandas as pd
import numpy as np 
import json
import ast
from collections import Counter
import xgboost as xgb
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
from sklearn.linear_model import Ridge
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import StackingRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from hyperopt import fmin, tpe, hp
from lightgbm import LGBMRegressor


# pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
DATA_FOLDER = "/StudentData/hw1/"

In [3]:
X_train_all = pd.read_csv(DATA_FOLDER + "X_train.csv")
y_train_all = pd.read_csv(DATA_FOLDER + "y_train.csv")
# X_train, X_valid, y_train, y_valid = train_test_split(X_train_all, y_train_all, test_size=0.33)
X_train = X_train_all
y_train = y_train_all
X_train

,log_budget,log_popularity,log_vote_average,log_vote_count,belongs_to_collection_A Nightmare on Elm Street Collection,belongs_to_collection_Friday the 13th Collection,belongs_to_collection_Halloween Collection,belongs_to_collection_James Bond Collection,belongs_to_collection_One Piece Collection,belongs_to_collection_Police Academy Collection,belongs_to_collection_Saw Collection,belongs_to_collection_The Pink Panther (Original) Collection,belongs_to_collection_The Terminator Collection,genres_Action,genres_Adventure,genres_Animation,genres_Comedy,genres_Crime,genres_Documentary,genres_Drama,genres_Family,genres_Fantasy,genres_History,genres_Horror,genres_Music,genres_Mystery,genres_Romance,genres_Science Fiction,genres_TV Movie,genres_Thriller,genres_War,genres_Western,production_companies_2,production_companies_4,production_companies_5,production_companies_7,production_companies_12,production_companies_14,production_companies_21,production_companies_25,production_companies_33,production_companies_34,production_companies_60,production_companies_79,production_companies_104,production_companies_174,production_companies_559,production_companies_694,production_companies_1632,production_companies_7295,production_companies_9195,production_companies_10163,production_countries_Australia,production_countries_Belgium,production_countries_Canada,production_countries_China,production_countries_Denmark,production_countries_France,production_countries_Germany,production_countries_Hong Kong,production_countries_India,production_countries_Ireland,production_countries_Italy,production_countries_Japan,production_countries_Mexico,production_countries_Netherlands,production_countries_Russia,production_countries_South Korea,production_countries_Spain,production_countries_Sweden,production_countries_United Kingdom,production_countries_United States of America,spoken_languages_ar,spoken_languages_cn,spoken_languages_cs,spoken_languages_de,spoken_languages_en,spoken_languages_es,spoken_languages_fr,spoken_languages_he,spoken_languages_hi,spoken_languages_hu,spoken_languages_it,spoken_languages_ja,spoken_languages_ko,spoken_languages_la,spoken_languages_pl,spoken_languages_pt,spoken_languages_ru,spoken_languages_sv,spoken_languages_tr,spoken_languages_zh,cast_Alec Baldwin,cast_Brad Pitt,cast_Bruce Willis,cast_Dennis Quaid,cast_Frank Welker,cast_J.K. Simmons,cast_John Goodman,cast_John Leguizamo,cast_Keith David,cast_Liam Neeson,cast_Matt Damon,cast_Morgan Freeman,cast_Nicolas Cage,cast_Richard Jenkins,cast_Robert De Niro,cast_Robin Williams,cast_Samuel L. Jackson,cast_Steve Buscemi,cast_Tommy Lee Jones,cast_Willem Dafoe,crew_Avy Kaufman,crew_Barbara Harris,crew_Bob Weinstein,crew_Clint Eastwood,crew_Dan O'Connell,crew_Dan Perri,crew_Deborah Aquila,crew_Gary Burritt,crew_Hans Zimmer,crew_Harvey Weinstein,crew_James Horner,crew_James Newton Howard,crew_John T. Cucci,crew_Kerry Barden,crew_Luc Besson,crew_Mary Vernieu,crew_Mo Henry,crew_Nerses Gezalyan,crew_Robert Rodriguez,crew_Steven Spielberg,original_language_ab,original_language_af,original_language_ar,original_language_bn,original_language_cn,original_language_cs,original_language_da,original_language_de,original_language_el,original_language_en,original_language_es,original_language_fa,original_language_fi,original_language_fr,original_language_he,original_language_hi,original_language_hu,original_language_id,original_language_it,original_language_ja,original_language_ka,original_language_ko,original_language_mi,original_language_ml,original_language_nb,original_language_nl,original_language_no,original_language_pl,original_language_pt,original_language_ro,original_language_ru,original_language_sk,original_language_sl,original_language_sv,original_language_sw,original_language_ta,original_language_te,original_language_th,original_language_tl,original_language_tr,original_language_vi,original_language_xx,original_language_zh,has_homepage,has_video,is_belongs_to_collection,title_length,overview_leng

In [4]:
def plot_prediction_stats(X_valid,y_valid,y_pred):
    # see where the model is failing so we can think of ways to improve it.
    pass

def adjust_prediction(pred):
    pred = np.e ** pred - 1
    pred[pred<0] = 0
    return pred


def cross_validation(model,X,y,num_folds,slient=True):
    rmlse_score = []
    folds = KFold(n_splits=num_folds, shuffle=True)
    for k, (train_indexes, valid_indexes) in enumerate(folds.split(X,y)):
        X_train, y_train = X.iloc[train_indexes], y.iloc[train_indexes]
        X_valid, y_valid = X.iloc[valid_indexes], y.iloc[valid_indexes]
        y_train = np.log(y_train + 1)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_valid)
        y_pred = adjust_prediction(y_pred)
        rmlse_fold = np.sqrt(mean_squared_log_error(y_valid, y_pred))
        rmlse_score.append(rmlse_fold)
        if not slient:
            print("RMLSE on fold {}: {}".format(k+1,rmlse_fold))
            plot_prediction_stats(X_valid,y_valid,y_pred)
    return np.mean(rmlse_score)

In [5]:
# model_xgb = xgb.XGBRegressor(max_depth=5, 
#                             learning_rate=0.01, 
#                             n_estimators=10000, 
#                             objective='reg:linear', 
#                             gamma=1.45, 
#                             silent=True,
#                             subsample=0.8, 
#                             colsample_bytree=0.7, 
#                             colsample_bylevel=0.5)
# X_train_scaled = pd.DataFrame(data=RobustScaler().fit_transform(X_train), columns=X_train.columns) # see one hot
# model_xgb = xgb.XGBRegressor()
# cross_validation(model_xgb,X_train_scaled,y_train,5)

In [6]:
# estimators = [('RandomForestRegressor', RandomForestRegressor()), ('XGBRegressor', xgb.XGBRegressor()), ('CatBoostRegressor', CatBoostRegressor(silent=True)), ('LGBMRegressor', LGBMRegressor())]
# model_stack = StackingRegressor(estimators=estimators,final_estimator=RandomForestRegressor())
# X_train = pd.DataFrame(data=RobustScaler().fit_transform(X_train), columns=X_train.columns)
# reg.fit(X_train, y_train).score(X_valid, y_test)
# for model_str, model in estimators:
#     print(model_str)
#     cross_validation(model,X_train,y_train,3,False)
# cross_validation(model_stack,X_train,y_train,3,False)

In [7]:
# RandomForestRegressor
params_rf = {'n_jobs': 4,
          'random_state': 0,
          'n_estimators': hp.choice('n_estimators', [100, 250, 500, 750, 1000]),
          'max_depth': hp.choice('max_depth', np.arange(3, 15, dtype=int)),
          'max_features': hp.choice('max_features', ['auto', 'sqrt', 'log2']),
          'min_samples_split': hp.uniform('min_samples_split', 0.01, 0.5),
          'min_samples_leaf': hp.uniform('min_samples_leaf', 0.01, 0.5)}

# XGBRegressor
params_xgb = {'silent': True,
          'objective': 'reg:linear',
          'seed': 0,
          'n_estimators': hp.choice('n_estimators', [100, 250, 500, 750, 1000]),
          'max_depth': hp.choice('max_depth', np.arange(3, 15, dtype=int)),
          'learning_rate': hp.uniform('learning_rate', 0.01, 0.75),  
          'colsample_bytree': hp.uniform('colsample_bytree', 0.1, 1)}

# CatBoostRegressor
params_cb = {'random_seed': 0,
          'early_stopping_rounds': 500,
          'eval_metric': 'RMSE',
          'logging_level': 'Silent',
          'n_estimators': 1000,
          'depth': hp.choice('max_depth', np.arange(3, 15, dtype=int)),
          'learning_rate': hp.uniform('learning_rate', 0.01, 0.75),
          'colsample_bylevel': hp.uniform('colsample_bylevel', 0.1, 1)}

# LGBMRegressor
params_lgbm = {'seed': 0,
          'tree_learner_type': hp.choice('tree_learner_type', ['serial', 'feature', 'data', 'voting']),
          'n_estimators': hp.choice('n_estimators', [100, 250, 500, 750, 1000]),
          'num_leaves': hp.choice('num_leaves', [50, 100, 150, 200, 250, 500, 750, 1000]),
          'min_data_in_leaf' : hp.choice('min_data_in_leaf', [10, 20, 30, 40, 50, 100]),
          'max_depth': hp.choice('max_depth', np.arange(3, 15, dtype=int)),
          'learning_rate': hp.uniform('learning_rate', 0.01, 0.75),}

In [8]:
def model_tunning(model_ctor,X,y,num_folds,slient=True):
    def internal_tuned(args):
        model = model_ctor(**args)
        return cross_validation(model,X,y,num_folds,slient)
    return internal_tuned

In [ ]:
models_params = [("RandomForestRegressor",RandomForestRegressor,params_rf),("XGBRegressor",xgb.XGBRegressor,params_xgb),("CatBoostRegressor",CatBoostRegressor,params_cb),("LGBMRegressor",LGBMRegressor,params_lgbm)]
estimators = []
for i, (model_str, model_ctor, params) in enumerate(models_params):
    best_params = fmin(model_tunning(model_ctor,X_train,y_train,4), params, algo=tpe.suggest, max_evals=20)
    estimators.append((model_str,model_ctor(**best_params),best_params))

  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

/home/student/miniconda3/envs/hw1/lib/python3.7/site-packages/ipykernel_launcher.py:18: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

/home/student/miniconda3/envs/hw1/lib/python3.7/site-packages/ipykernel_launcher.py:18: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

/home/student/miniconda3/envs/hw1/lib/python3.7/site-packages/ipykernel_launcher.py:18: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

/home/student/miniconda3/envs/hw1/lib/python3.7/site-packages/ipykernel_launcher.py:18: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



 10%|█         | 1/10 [00:01<00:12,  1.37s/trial, best loss: 2.7893487393596557]

/home/student/miniconda3/envs/hw1/lib/python3.7/site-packages/ipykernel_launcher.py:18: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

/home/student/miniconda3/envs/hw1/lib/python3.7/site-packages/ipykernel_launcher.py:18: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

/home/student/miniconda3/envs/hw1/lib/python3.7/site-packages/ipykernel_launcher.py:18: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

/home/student/miniconda3/envs/hw1/lib/python3.7/site-packages/ipykernel_launcher.py:18: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



 20%|██        | 2/10 [00:07<00:22,  2.76s/trial, best loss: 2.7893487393596557]

...

 80%|████████  | 8/10 [00:25<00:05,  2.90s/trial, best loss: 2.4552163386664323]

/home/student/miniconda3/envs/hw1/lib/python3.7/site-packages/ipykernel_launcher.py:18: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

/home/student/miniconda3/envs/hw1/lib/python3.7/site-packages/ipykernel_launcher.py:18: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

/home/student/miniconda3/envs/hw1/lib/python3.7/site-packages/ipykernel_launcher.py:18: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

/home/student/miniconda3/envs/hw1/lib/python3.7/site-packages/ipykernel_launcher.py:18: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



 90%|█████████ | 9/10 [00:31<00:03,  3.78s/trial, best loss: 2.4552163386664323]

/home/student/miniconda3/envs/hw1/lib/python3.7/site-packages/ipykernel_launcher.py:18: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

/home/student/miniconda3/envs/hw1/lib/python3.7/site-packages/ipykernel_launcher.py:18: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

/home/student/miniconda3/envs/hw1/lib/python3.7/site-packages/ipykernel_launcher.py:18: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

/home/student/miniconda3/envs/hw1/lib/python3.7/site-packages/ipykernel_launcher.py:18: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

In [15]:
# stack_reg = StackingRegressor(estimators=estimators,final_estimator=RandomForestRegressor())
for model_str, model in estimators[1:]:
    cross_validation(model,X_valid,y_valid,4)
# cross_validation(estimators[4][1],X_valid,y_valid,4)

0:	learn: 3.0890203	total: 1.93ms	remaining: 1.93s
1:	learn: 3.0801623	total: 3.69ms	remaining: 1.84s
2:	learn: 3.0707273	total: 5.74ms	remaining: 1.91s
3:	learn: 3.0603831	total: 7.67ms	remaining: 1.91s
4:	learn: 3.0491064	total: 9.55ms	remaining: 1.9s
5:	learn: 3.0400120	total: 11.4ms	remaining: 1.89s
6:	learn: 3.0307962	total: 13.3ms	remaining: 1.89s
7:	learn: 3.0212661	total: 15.2ms	remaining: 1.88s
8:	learn: 3.0124693	total: 17.1ms	remaining: 1.89s
9:	learn: 3.0038599	total: 19ms	remaining: 1.88s
10:	learn: 2.9943972	total: 20.8ms	remaining: 1.87s
11:	learn: 2.9864508	total: 22.8ms	remaining: 1.88s
12:	learn: 2.9777235	total: 24.7ms	remaining: 1.87s
13:	learn: 2.9686452	total: 26.5ms	remaining: 1.86s
14:	learn: 2.9603815	total: 28.7ms	remaining: 1.88s
15:	learn: 2.9507122	total: 30.7ms	remaining: 1.89s
16:	learn: 2.9418866	total: 32.7ms	remaining: 1.89s
17:	learn: 2.9337452	total: 34.5ms	remaining: 1.88s
18:	learn: 2.9251698	total: 36.3ms	remaining: 1.87s
19:	learn: 2.9169313	tota

ValueError: num_boost_round should be greater than zero.

In [14]:
print(estimators)

[('RandomForestRegressor', RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=10, max_features=0, max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None,
                      min_samples_leaf=0.06455593349387313,
                      min_samples_split=0.13714728425816008,
                      min_weight_fraction_leaf=0.0, n_estimators=4, n_jobs=None,
                      oob_score=False, random_state=None, verbose=0,
                      warm_start=False)), ('XGBRegressor', XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=0.47574783802147225,
             gamma=None, gpu_id=None, importance_type='gain',
             interaction_constraints=None, learning_rate=0.3817191032398365,
             max_delta_step=None, max_depth=0, min_child_weight=None,
             missing=nan, monotone_

In [7]:
np.arange(3, 30, 2, dtype=int)

array([ 3,  5,  7,  9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29])